In [ ]:
!pip install open-clip-torch
!pip install jaxtyping einops open-clip-torch ftfy regex timm

## Import delle librerie necessarie

In [ ]:
import os,json
from pathlib import Path

import pandas as pd
from PIL import Image

import torch
import torch.nn.functional as F
from sphinx.ext.viewcode import OUTPUT_DIRNAME
from tqdm import tqdm

#Todo togliere il commmento
import open_clip


Impostazione del dispositivo di calcolo

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


Clip_model per ottenere gli score delle generazioni

In [ ]:
#Inizializzazione CLIP (per score)
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
clip_model= clip_model.to(device).eval()
clip_tokenizer=open_clip.get_tokenizer('ViT-B-32')

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Definizioni dei Path


In [ ]:
from pathlib import Path

BASE_DATA_DIR=Path("/content")
CSV_PATH=BASE_DATA_DIR/"dataset/cambio_materiale/change_material_tests.csv"
OUT_DIR=Path("/content/outputs_change_materials")
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('BASE_DATA_DIR =', BASE_DATA_DIR)
print('CSV_PATH      =', CSV_PATH)
print('OUT_DIR       =', OUT_DIR)

BASE_DATA_DIR = /content
CSV_PATH      = /content/dataset/cambio_materiale/change_material_tests.csv
OUT_DIR       = /content/outputs_change_materials


Import del Token-Opt


In [ ]:
import sys
!git clone https://github.com/lukaslaobeyer/token-opt.git /content/token-opt
REPO_DIR = Path('/content/token-opt')
sys.path.insert(0, str(REPO_DIR))

TTO_AVAILABLE = True
try:
    from tto.test_time_opt import(
        TestTimeOpt,
        TestTimeOptConfig,
        CLIPObjective,
    )
    print("Token-Opt importato correttamente")
except Exception as e:
    TTO_AVAILABLE = False
    print("Errore nell'importazione di Token-Opt:", e)

print(TTO_AVAILABLE)


fatal: destination path '/content/token-opt' already exists and is not an empty directory.
Token-Opt importato correttamente
True


Definizione di caricamento immagine e funzione di calcolo score CLIP

In [ ]:
def load_rgb_image(path:Path):
    image = Image.open(path).convert("RGB")
    return image

def clip_score_image_text(img:Image.Image,text:str) -> float:
    with torch.no_grad():
        image_input=clip_preprocess(img).unsqueeze(0).to(device)
        text_input=clip_tokenizer([text]).to(device)

        image_feature=clip_model.encode_image(image_input)
        text_feature=clip_model.encode_text(text_input)

        image_feature=F.normalize(image_feature,dim=-1)
        text_feature=F.normalize(text_feature,dim=-1)

        sim=(image_feature@text_feature.T).squeeze().item()
    return float(sim)

Per trasformare immagine in tensore e tensore in immagine

In [ ]:
import torchvision.transforms as T
_pre_tto = T.Compose([
    T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(256),
    T.ToTensor(),  # -> [0,1] float32, shape [3,H,W]
])

def pil_to_tto(seed_img: Image.Image, device: str = "cuda") -> torch.Tensor:
    x = _pre_tto(seed_img.convert("RGB"))     # [3,H,W] in [0,1]
    x = x.unsqueeze(0).to(device, dtype=torch.float32)  # [1,3,H,W]
    return x
def tensor_to_pil(img: torch.Tensor) -> Image.Image:
    # accetta [1,3,H,W] o [3,H,W]; assume range [0,1]
    if img.ndim == 4:
        img = img[0]
    img = img.detach().clamp(0, 1).cpu()
    img = (img * 255).to(torch.uint8)      # [3,H,W]
    img = img.permute(1, 2, 0).numpy()     # HWC
    return Image.fromarray(img)

# Metodo di generazione dell'immagine

In [ ]:
def run_token_opt(seed_img:Image.Image,prompt:str,steps:int=50)->Image.Image:
    """
    Esegue Token_Opt su una singola immagine+prompt. Se Token-Opt non è importato correttamente si restituisce l'immagine seed.
    """
    if not TTO_AVAILABLE:
        print("Token-Opt non è disponibile, restituisco l'immagine seed")
        return seed_img

    seed_tensor=pil_to_tto(seed_img,device="cuda")

    cfg=TestTimeOptConfig(
        num_iter=steps,
        lr=0.1,
    )

    #Definizione dell'obiettivo CLIP
    objective=CLIPObjective(prompt=prompt)

    tto=TestTimeOpt(cfg,objective).to(device=device)
    torch.manual_seed(0)

    tensor_out=tto(seed=seed_tensor)

    return tensor_to_pil(tensor_out)


# Loop principale

In [ ]:
df=pd.read_csv(CSV_PATH) #Lettura del csv

required_cols=['id','macrocat','scenario','object','input_image_path','prompt_target']

missing=[col for col in required_cols if col not in df.columns]
if missing:
    raise ValueError(f"Mancano le seguenti colonne nel CSV: {missing}")
print("Totale righe da processare",len(df))
results=[]

for _,row in tqdm(df.iterrows(),total=len(df)):
    test_id=row['id']
    scenario=row['scenario']
    obj=row['object']
    rel_path=row['input_image_path']
    prompt=row['prompt_target']

    img_path=BASE_DATA_DIR/Path(rel_path)
    print(img_path)
    if not img_path.exists():
        print(f"[WARN] immagine mancante {img_path}")
        continue
    out_dir=OUT_DIR/str(row['macrocat']) /str(obj)/str(scenario)/str(test_id)
    out_dir.mkdir(parents=True,exist_ok=True)

    (out_dir/"prompt.txt").write_text(prompt,encoding='utf-8')

    seed_image=load_rgb_image(img_path)
    seed_image.save(out_dir/"input.jpg")

    #Generazione immagine
    image_out=run_token_opt(seed_image,prompt,steps=200)
    out_image_path=out_dir/"output.jpg"
    image_out.save(out_image_path)

    #CLIP score
    score=clip_score_image_text(image_out,prompt)

    #Metadata JSON
    meta = {
        'id': test_id,
        'macrocat': row['macrocat'],
        'scenario': scenario,
        'object': obj,
        'prompt': prompt,
        'input_image_path': str(img_path),
        'output_image_path': str(out_image_path),
        'clip_score': score,
    }
    with open(out_dir / 'clip_scores.json', 'w', encoding='utf-8') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    results.append(meta)

res_df = pd.DataFrame(results)
res_csv = OUT_DIR / 'results_clip.csv'
res_df.to_csv(res_csv, index=False)
print('[DONE] Salvato results in:', res_csv)

if len(results):
    res_df = pd.DataFrame(results)
    print('Media CLIP per (object, scenario):')
    display(res_df.groupby(['object','scenario'])['clip_score'].mean().round(4))

    print('\nTop 5 risultati per clip_score:')
    display(res_df.sort_values('clip_score', ascending=False).head(5)[['id','object','scenario','clip_score','prompt']])

    print('\nBottom 5 risultati per clip_score:')
    display(res_df.sort_values('clip_score', ascending=True).head(5)[['id','object','scenario','clip_score','prompt']])
else:
    print('Nessun risultato: controlla path immagini / CSV.')



Totale righe da processare 30


  0%|          | 0/30 [00:00<?, ?it/s]

/content/dataset/cambio_materiale/clean/sedia.jpg


  3%|▎         | 1/30 [01:53<54:40, 113.11s/it]

/content/dataset/cambio_materiale/real/sedia.jpg


...

object  scenario
chair   clean       0.3100
        real        0.2991
cup     clean       0.3223
        real        0.3298
lamp    clean       0.2945
        real        0.3026
sofa    clean       0.2877
        real        0.3174
table   clean       0.2947
        real        0.3270
vase    clean       0.3391
        real        0.3275
Name: clip_score, dtype: float64


Top 5 risultati per clip_score:


,id,object,scenario,clip_score,prompt
15,MAT_016,table,real,0.349721,Transform the table into a transparent glass t...
23,MAT_022,vase,real,0.349568,Transform the terracotta vase into a natural s...
22,MAT_021,vase,clean,0.349549,Transform the terracotta vase into a natural s...
9,MAT_010,sofa,real,0.337466,Make the sofa appear as made of glossy plastic...
10,MAT_011,table,clean,0.335973,Transform the table into a polished metal tabl...



Bottom 5 risultati per clip_score:


,id,object,scenario,clip_score,prompt
6,MAT_007,sofa,clean,0.257873,Turn the fabric sofa into a brown brown leathe...
12,MAT_013,table,clean,0.258209,Transform the table into a polished marble tab...
5,MAT_006,chair,real,0.280109,Transform the chair so that it is with a trans...
16,MAT_029,table,clean,0.288087,Transform the table into a natural stone table...
28,MAT_027,lamp,clean,0.293317,Transform the lamp head into transparent glass...


In [ ]:
!zip -r outputs_change_materials-zip /content/outputs_change_materials

updating: content/outputs_change_materials/ (stored 0%)
updating: content/outputs_change_materials/.ipynb_checkpoints/ (stored 0%)
updating: content/outputs_change_materials/change_material/ (stored 0%)
updating: content/outputs_change_materials/change_material/cup/ (stored 0%)
updating: content/outputs_change_materials/change_material/cup/real/ (stored 0%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_018/ (stored 0%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_018/prompt.txt (deflated 23%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_018/clip_scores.json (deflated 43%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_018/input.jpg (deflated 1%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_018/output.jpg (deflated 2%)
updating: content/outputs_change_materials/change_material/cup/real/MAT_020/ (stored 0%)
updating: content/outputs_change_materials/change_m